# Poem Polarity Detection

In [1]:
import sys
relative_path = "../../"
sys.path.append(relative_path)
import os
from lookup import LookupCreator
from sensepolar.polarity import WordPolarity
from sensepolar.embed.bertEmbed import BERTWordEmbeddings
from sensepolar.embed.albertEmbed import ALBERTWordEmbeddings
from sensepolar.embed.robertaEmbed import RoBERTaWordEmbeddings
from sensepolar.polarDim import PolarDimensions
from sensepolar.oracle.dictionaryapi import Dictionary
import nltk
import pandas as pd
from nltk.stem import PorterStemmer
from datasets import load_dataset
from datasets import Dataset as Data
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import numpy as np
from transformers import BertTokenizer, BertModel
from IPython.utils import io
import re
import torch.nn.utils.rnn as rnn_utils

## BERT Word Embeddings

In [16]:
# dataset = load_dataset("poem_sentiment")
# train = dataset["train"]
# valid = dataset["validation"]
# test = dataset["test"]
train =  Data.from_pandas(pd.read_csv('data/train.tsv', sep='\t')) 
valid = Data.from_pandas(pd.read_csv('data/dev.tsv', sep='\t')) 
test = Data.from_pandas(pd.read_csv('data/test.tsv', sep='\t')) 

#### Data Fields

    id: index of the example
    verse_text: The text of the poem verse
    label: The sentiment label. Here
        0 = negative
        1 = positive
        2 = no impact
        3 = mixed (both negative and positive)

In [ ]:
train_df = pd.DataFrame(train)
train_df

,0,with pale blue berries. in these peaceful shades--,1
0,1,"it flows so long as falls the rain,",0
1,2,"and that is why, the lonesome day,",-1
2,3,"when i peruse the conquered fame of heroes, an...",2
3,4,of inward strife for truth and liberty.,2
4,5,the red sword sealed their vows!,2
...,...,...,...
743,887,to his ears there came a murmur of far seas be...,0
744,888,"the one good man in the world who knows me, --",1
745,889,faint voices lifted shrill with pain,-1
746,890,"an', fust you knowed on, back come charles the...",0


In [17]:
valid_df = pd.DataFrame(valid)
valid_df

,id,verse_text,label
0,0,"to water, cloudlike on the bush afar,",2
1,1,"shall yet be glad for him, and he shall bless",1
2,2,on its windy site uplifting gabled roof and pa...,2
3,3,(if haply the dark will of fate,0
4,4,"jehovah, jove, or lord!",2
...,...,...,...
100,100,"said my companion, 'i will show you soon",2
101,101,but god said,2
102,102,but if thou do thy best,2
103,103,"so generous to me. farewell, friend, since friend",1


In [18]:
test_df = pd.DataFrame(test)
test_df

,id,verse_text,label
0,0,"my canoe to make more steady,",2
1,1,and be glad in the summer morning when the kin...,1
2,2,and when they reached the strait symplegades,2
3,3,she sought for flowers,2
4,4,"if they are hungry, paradise",2
...,...,...,...
99,99,shall live my highland mary.,2
100,100,now is past since last we met,2
101,101,begins to live,2
102,102,beneath the hazel bough;,2


In [ ]:
class PoemSentimentDataset(Dataset):
    def __init__(self, verse_text, labels, tokenizer, max_length):
        self.verse_text = verse_text
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.verse_text)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.verse_text[idx],
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        label = torch.tensor(self.labels[idx], dtype=torch.long)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': label
        }


In [ ]:
class BERTEmbeddingClassifier(nn.Module):
    def __init__(self, num_classes, bert_model_name='bert-base-uncased'):
        super(BERTEmbeddingClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        word_embeddings = self.bert(input_ids=input_ids, attention_mask=attention_mask)[0]
        output = self.dropout(word_embeddings[:, 0, :])  # Use the [CLS] token representation
        logits = self.fc(output)
        return logits

    def train_model(self, train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device):
        best_valid_loss = float('inf')
        epochs_without_improvement = 0

        for epoch in range(num_epochs):
            self.train()
            train_loss = 0.0
            for batch in train_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)

                optimizer.zero_grad()

                outputs = self(input_ids, attention_mask=attention_mask)
                logits = outputs
                loss = loss_fn(logits, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()

            avg_train_loss = train_loss / len(train_loader)
            print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {avg_train_loss}")

            # Validation
            self.eval()
            valid_loss = 0.0
            with torch.no_grad():
                for batch in valid_loader:
                    input_ids = batch['input_ids'].to(device)
                    attention_mask = batch['attention_mask'].to(device)
                    labels = batch['label'].to(device)

                    outputs = self(input_ids, attention_mask=attention_mask)
                    logits = outputs
                    loss = loss_fn(logits, labels)

                    valid_loss += loss.item()

            avg_valid_loss = valid_loss / len(valid_loader)
            print(f"Epoch {epoch+1}/{num_epochs} - Validation Loss: {avg_valid_loss}")

            if avg_valid_loss < best_valid_loss:
                best_valid_loss = avg_valid_loss
                epochs_without_improvement = 0
                torch.save(self.state_dict(), "model/trained_model.pth")
            else:
                epochs_without_improvement += 1
                if epochs_without_improvement >= patience:
                    print(f"Early stopping. No improvement in {patience} epochs.")
                    break

        self.load_state_dict(torch.load("model/trained_model.pth"))

    def test_model(self, test_loader, loss_fn, device):
        self.eval()
        test_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        with torch.no_grad():
            for batch in test_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)

                outputs = self(input_ids, attention_mask=attention_mask)
                logits = outputs
                loss = loss_fn(logits, labels)
                test_loss += loss.item()

                _, predicted = torch.max(logits, dim=1)
                correct_predictions += (predicted == labels).sum().item()
                total_samples += labels.size(0)

        avg_test_loss = test_loss / len(test_loader)
        accuracy = correct_predictions / total_samples
        print(f"Test Loss: {avg_test_loss}, Accuracy: {accuracy}")

    def predict(self, input_text, tokenizer, max_length, device):
        inputs = tokenizer.encode_plus(
            input_text,
            add_special_tokens=True,
            truncation=True,
            max_length=max_length,
            padding='max_length',
            return_tensors='pt'
        )

        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)

        self.eval()
        with torch.no_grad():
            outputs = self(input_ids, attention_mask=attention_mask)
            logits = outputs

        _, predicted = torch.max(logits, dim=1)
        return predicted.item()


In [28]:
dataset = load_dataset("poem_sentiment")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
embedding_model = BertModel.from_pretrained('bert-base-uncased')

num_classes = 4
bert_trained_model = BERTEmbeddingClassifier(num_classes=num_classes, bert_model_name='bert-base-uncased')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
bert_trained_model.to(device)

optimizer = torch.optim.AdamW(bert_trained_model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 50
patience = 4

preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]
valid_labels = dataset["validation"]["label"]

# train_texts_filtered = []
# train_labels_filtered = []
# for text, label in zip(train_texts, train_labels):
#     if label != 3:
#         train_texts_filtered.append(text)
#         train_labels_filtered.append(label)

train_dataset = PoemSentimentDataset(train_texts, train_labels, tokenizer, 128)
valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, tokenizer, 128)
test_dataset = PoemSentimentDataset(test_texts, test_labels, tokenizer, 128)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

bert_trained_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)


bert_trained_model.test_model(test_loader, loss_fn, device)

Using custom data configuration default
Reusing dataset poem_sentiment (/home/ma/ma_ma/ma_prajula/.cache/huggingface/datasets/poem_sentiment/default/1.0.0/4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099)


  0%|          | 0/3 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bi

cpu
Epoch 1/50 - Training Loss: 1.0132639716778482
Epoch 1/50 - Validation Loss: 0.6547691907201495
Epoch 2/50 - Training Loss: 0.5602611228823662
Epoch 2/50 - Validation Loss: 0.41771547496318817
Epoch 3/50 - Training Loss: 0.2249130530149809
Epoch 3/50 - Validation Loss: 0.4881617586527552
Epoch 4/50 - Training Loss: 0.10362077860294708
Epoch 4/50 - Validation Loss: 0.6068014992134911
Epoch 5/50 - Training Loss: 0.056983754970133305
Epoch 5/50 - Validation Loss: 0.6851357498339244
Epoch 6/50 - Training Loss: 0.02596069744322449
Epoch 6/50 - Validation Loss: 0.6746343182665961
Early stopping. No improvement in 4 epochs.
Test Loss: 0.38950822608811514, Accuracy: 0.8365384615384616


#### BERT Word Embeddings

Epoch 1/50 - Training Loss: 0.8396447778872724
Epoch 1/50 - Validation Loss: 0.6666120759078434
Epoch 2/50 - Training Loss: 0.37625285181796775
Epoch 2/50 - Validation Loss: 0.3860052094927856
Epoch 3/50 - Training Loss: 0.10331153671063904
Epoch 3/50 - Validation Loss: 0.6297259892203978
Epoch 4/50 - Training Loss: 0.049240260745205405
Epoch 4/50 - Validation Loss: 0.5406883760754552
Epoch 5/50 - Training Loss: 0.016198201860599924
Epoch 5/50 - Validation Loss: 0.6031707083540303
Epoch 6/50 - Training Loss: 0.007132284079660785
Epoch 6/50 - Validation Loss: 0.6020233376350786
Early stopping. No improvement in 4 epochs.
Test Loss: 0.3860052094927856, Accuracy: 0.8516746411483254

## SensePOLAR Embeddings

In [2]:
class PoemSentimentDataset(Dataset):
    def __init__(self, verse_text, labels, word_polarity_model, method='cls', dimension=39):
        self.verse_text = verse_text
        self.labels = labels
        self.word_polarity_model = word_polarity_model
        self.polar_embeddings_cache = {}
        self.dimension = dimension
        self.method = method

    def __len__(self):
        return len(self.verse_text)

    def get_sense_polar_embedding(self, word, context):
        if (word, context) not in self.polar_embeddings_cache:
            with io.capture_output() as captured:
                polar_embedding = self.word_polarity_model.analyze_word(word, context)
            antonym_dict = {}
            for pair in polar_embedding:
                antonym_dict[(pair[0], pair[1])] = antonym_dict.get((pair[0], pair[1]), []) + [pair[2]]
            sorted_antonym_dict = dict(sorted(antonym_dict.items(), key=lambda item: item[0]))
            self.polar_embeddings_cache[(word, context)] = list(sorted_antonym_dict.values())
        return self.polar_embeddings_cache[(word, context)]

    def __getitem__(self, idx):
        verse = self.verse_text[idx]
        labels = self.labels[idx]
        verse_polar_embeddings = None

        if self.method == 'cls':
            verse += ' [CLS]'
            cls_polar_embedding = self.get_sense_polar_embedding('[CLS]', verse)
            verse_polar_embeddings = torch.tensor(cls_polar_embedding, dtype=torch.float)
        else:
            polar_embeddings_list = []
            for word in verse.split():
                polar_embedding = self.get_sense_polar_embedding(word, verse)
                polar_embeddings_list.append(polar_embedding)
            verse_polar_embeddings = torch.tensor(polar_embeddings_list, dtype=torch.float)
            verse_polar_embeddings = torch.mean(verse_polar_embeddings, dim=0)
            
        verse_polar_embeddings = verse_polar_embeddings.long()
        verse_polar_embeddings = verse_polar_embeddings.squeeze(dim=1)
        verse_polar_embeddings = verse_polar_embeddings[:self.dimension]
        label = torch.tensor(labels, dtype=torch.long)

        return {
            'polar_embeddings': verse_polar_embeddings,
            'label': label
        }


In [3]:

class PolarEmbeddingClassifier(nn.Module):
    def __init__(self, num_classes, polar_dimension, model_name='sense_polar_model.pth'):
        super(PolarEmbeddingClassifier, self).__init__()
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(polar_dimension, num_classes)
        self.model_name = model_name

    def forward(self, polar_embeddings):
        output = self.dropout(polar_embeddings.float())
        logits = self.fc(output)
        return logits

    def train_model(self, train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device):
        best_valid_loss = float('inf')
        epochs_without_improvement = 0

        for epoch in range(num_epochs):
            self.train()
            train_loss = 0.0
            for batch in train_loader:
                polar_embeddings = batch['polar_embeddings'].to(device)
                labels = batch['label'].to(device)

                optimizer.zero_grad()

                outputs = self(polar_embeddings)  
                logits = outputs
                loss = loss_fn(logits, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()

            avg_train_loss = train_loss / len(train_loader)
            print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {avg_train_loss}")

            # Validation
            self.eval()
            valid_loss = 0.0
            with torch.no_grad():
                for batch in valid_loader:
                    polar_embeddings = batch['polar_embeddings'].to(device)  
                    labels = batch['label'].to(device)

                    outputs = self(polar_embeddings)
                    logits = outputs
                    loss = loss_fn(logits, labels)

                    valid_loss += loss.item()

            avg_valid_loss = valid_loss / len(valid_loader)
            print(f"Epoch {epoch+1}/{num_epochs} - Validation Loss: {avg_valid_loss}")

            if avg_valid_loss < best_valid_loss:
                best_valid_loss = avg_valid_loss
                epochs_without_improvement = 0
                torch.save(self.state_dict(), "model/" + self.model_name)
            else:
                epochs_without_improvement += 1
                if epochs_without_improvement >= patience:
                    print(f"Early stopping. No improvement in {patience} epochs.")
                    break

        self.load_state_dict(torch.load("model/" + self.model_name))

    def test_model(self, test_loader, loss_fn, device):
        self.eval()
        test_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        with torch.no_grad():
            for batch in test_loader:
                polar_embeddings = batch['polar_embeddings'].to(device)  # Use 'polar_embeddings' key
                labels = batch['label'].to(device)

                outputs = self(polar_embeddings)  # Pass the polar_embeddings directly to the model
                logits = outputs
                loss = loss_fn(logits, labels)
                test_loss += loss.item()

                _, predicted = torch.max(logits, dim=1)
                correct_predictions += (predicted == labels).sum().item()
                total_samples += labels.size(0)

        avg_test_loss = test_loss / len(test_loader)
        accuracy = correct_predictions / total_samples
        print(f"Test Loss: {avg_test_loss}, Accuracy: {accuracy}")

    def predict(self, polar_embeddings, device):  # Updated predict method to use polar_embeddings directly
        polar_embeddings = polar_embeddings.to(device)
        
        self.eval()
        with torch.no_grad():
            outputs = self(polar_embeddings)
            logits = outputs

        _, predicted = torch.max(logits, dim=1)
        return predicted.item()


### SensePolar-BERT-CLS-39 dimensions

In [7]:
with io.capture_output() as captured:
    dataset = load_dataset("poem_sentiment")
    embedding_model_name = 'bert-base-uncased'
    out_path = './antonyms/'
    antonym_path = "data/polars.xlsx"
    embed_model = BERTWordEmbeddings()

    dictionary = Dictionary('wordnet', api_key='')    
    lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
    lookupSpace.create_lookup_files()
    antonym_path = out_path + "polar_dimensions.pkl"

    pdc = PolarDimensions(embed_model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
    pdc.create_polar_dimensions(out_path)

    wp = WordPolarity(embed_model, antonym_path=antonym_path, method='projection')
    num_classes = 3
    bert_sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=39, model_name='sense_polar_bert_39dim_cls_projection.pth')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_sensepolar_model.to(device)

optimizer = torch.optim.AdamW(bert_sensepolar_model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 1000
patience = 10

preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]
valid_labels = dataset["validation"]["label"]

train_texts_filtered = []
train_labels_filtered = []
for text, label in zip(train_texts, train_labels):
    if label != 3:
        train_texts_filtered.append(text)
        train_labels_filtered.append(label)

train_dataset = PoemSentimentDataset(train_texts_filtered, train_labels_filtered, wp)
valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp)
test_dataset = PoemSentimentDataset(test_texts, test_labels, wp)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

bert_sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

bert_sensepolar_model.test_model(test_loader, loss_fn, device)

Epoch 1/1000 - Training Loss: 1.134737085621312
Epoch 1/1000 - Validation Loss: 0.952536301953452
Epoch 2/1000 - Training Loss: 0.9253856359787707
Epoch 2/1000 - Validation Loss: 0.9083268983023507
Epoch 3/1000 - Training Loss: 0.9176082880991809
Epoch 3/1000 - Validation Loss: 0.8950522031102862
Epoch 4/1000 - Training Loss: 0.8896940721655792
Epoch 4/1000 - Validation Loss: 0.8832765306745257
Epoch 5/1000 - Training Loss: 0.8835140095566804
Epoch 5/1000 - Validation Loss: 0.8721444777079991
Epoch 6/1000 - Training Loss: 0.881023959168848
Epoch 6/1000 - Validation Loss: 0.8640147703034537
Epoch 7/1000 - Training Loss: 0.8791150810583582
Epoch 7/1000 - Validation Loss: 0.8514932819775173
Epoch 8/1000 - Training Loss: 0.8746731461219068
Epoch 8/1000 - Validation Loss: 0.8442184243883405
Epoch 9/1000 - Training Loss: 0.8540326008256877
Epoch 9/1000 - Validation Loss: 0.8344356077057975
Epoch 10/1000 - Training Loss: 0.8556005190003593
Epoch 10/1000 - Validation Loss: 0.826416552066803
Ep

#### BERT- SENSEPOLAR- AVERAGE - 39 DIM

In [8]:
with io.capture_output() as captured:
    dataset = load_dataset("poem_sentiment")
    embedding_model_name = 'bert-base-uncased'
    out_path = './antonyms/'
    antonym_path = "data/polars.xlsx"
    embed_model = BERTWordEmbeddings()

    dictionary = Dictionary('wordnet', api_key='')    
    lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
    lookupSpace.create_lookup_files()
    antonym_path = out_path + "polar_dimensions.pkl"

    pdc = PolarDimensions(embed_model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
    pdc.create_polar_dimensions(out_path)

    wp = WordPolarity(embed_model, antonym_path=antonym_path, method='projection')
    num_classes = 3
    bert_sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=39, model_name='sense_polar_bert_39dim_avg_projection.pth')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_sensepolar_model.to(device)

optimizer = torch.optim.AdamW(bert_sensepolar_model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 1000
patience = 10

preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]
valid_labels = dataset["validation"]["label"]

train_texts_filtered = []
train_labels_filtered = []
for text, label in zip(train_texts, train_labels):
    if label != 3:
        train_texts_filtered.append(text)
        train_labels_filtered.append(label)

train_dataset = PoemSentimentDataset(train_texts_filtered, train_labels_filtered, wp)
valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp)
test_dataset = PoemSentimentDataset(test_texts, test_labels, wp)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

bert_sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

bert_sensepolar_model.test_model(test_loader, loss_fn, device)

Epoch 1/1000 - Training Loss: 0.9978979027496194
Epoch 1/1000 - Validation Loss: 0.9357996327536446
Epoch 2/1000 - Training Loss: 0.9161276412460039
Epoch 2/1000 - Validation Loss: 0.9185548850468227
Epoch 3/1000 - Training Loss: 0.9126412103760917
Epoch 3/1000 - Validation Loss: 0.9065491727420262
Epoch 4/1000 - Training Loss: 0.8979238417913329
Epoch 4/1000 - Validation Loss: 0.8952271682875497
Epoch 5/1000 - Training Loss: 0.886234701804395
Epoch 5/1000 - Validation Loss: 0.8818017414637974
Epoch 6/1000 - Training Loss: 0.8750990280565226
Epoch 6/1000 - Validation Loss: 0.871656426361629
Epoch 7/1000 - Training Loss: 0.8662823517367525
Epoch 7/1000 - Validation Loss: 0.862412086554936
Epoch 8/1000 - Training Loss: 0.8594710157727296
Epoch 8/1000 - Validation Loss: 0.8522793310029166
Epoch 9/1000 - Training Loss: 0.8537818688266682
Epoch 9/1000 - Validation Loss: 0.842484542301723
Epoch 10/1000 - Training Loss: 0.8532396937316319
Epoch 10/1000 - Validation Loss: 0.8367171628134591
Ep

## 15 dimensions

In [10]:
# with io.capture_output() as captured:
#     dataset = load_dataset("poem_sentiment")
#     embedding_model_name = 'bert-base-uncased'
#     out_path = './antonyms/'
#     antonym_path = "data/polars.xlsx"
#     embed_model = BERTWordEmbeddings()

#     dictionary = Dictionary('wordnet', api_key='')    
#     lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
#     lookupSpace.create_lookup_files()
#     antonym_path = out_path + "polar_dimensions.pkl"

#     pdc = PolarDimensions(embed_model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
#     pdc.create_polar_dimensions(out_path)

#     wp = WordPolarity(embed_model, antonym_path=antonym_path, method='projection')
#     num_classes = 3
bert_sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=15, model_name='sense_polar_bert_15dim_cls_projection.pth')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_sensepolar_model.to(device)

optimizer = torch.optim.AdamW(bert_sensepolar_model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 1000
patience = 10

preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]
valid_labels = dataset["validation"]["label"]

train_texts_filtered = []
train_labels_filtered = []
for text, label in zip(train_texts, train_labels):
    if label != 3:
        train_texts_filtered.append(text)
        train_labels_filtered.append(label)

train_dataset = PoemSentimentDataset(train_texts_filtered, train_labels_filtered, wp, dimension=15)
valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp, dimension=15)
test_dataset = PoemSentimentDataset(test_texts, test_labels, wp, dimension=15)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

bert_sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

bert_sensepolar_model.test_model(test_loader, loss_fn, device)

Epoch 1/1000 - Training Loss: 0.9406273365020752
Epoch 1/1000 - Validation Loss: 0.9196240816797528
Epoch 2/1000 - Training Loss: 0.938369907298178
Epoch 2/1000 - Validation Loss: 0.9119172862597874
Epoch 3/1000 - Training Loss: 0.9270883978537794
Epoch 3/1000 - Validation Loss: 0.907514248575483
Epoch 4/1000 - Training Loss: 0.9199140600438388
Epoch 4/1000 - Validation Loss: 0.903494622026171
Epoch 5/1000 - Training Loss: 0.9045726533206004
Epoch 5/1000 - Validation Loss: 0.8998903121267047
Epoch 6/1000 - Training Loss: 0.9061605435497356
Epoch 6/1000 - Validation Loss: 0.8959686415536063
Epoch 7/1000 - Training Loss: 0.9015618427744452
Epoch 7/1000 - Validation Loss: 0.893102731023516
Epoch 8/1000 - Training Loss: 0.8988501170896134
Epoch 8/1000 - Validation Loss: 0.8888820154326302
Epoch 9/1000 - Training Loss: 0.8899233487417113
Epoch 9/1000 - Validation Loss: 0.8858283502714974
Epoch 10/1000 - Training Loss: 0.8867998258122858
Epoch 10/1000 - Validation Loss: 0.8830528599875314
Ep

In [11]:
bert_sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=15, model_name='sense_polar_bert_15dim_avg_projection.pth')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_sensepolar_model.to(device)

optimizer = torch.optim.AdamW(bert_sensepolar_model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 1000
patience = 10

preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]
valid_labels = dataset["validation"]["label"]

train_texts_filtered = []
train_labels_filtered = []
for text, label in zip(train_texts, train_labels):
    if label != 3:
        train_texts_filtered.append(text)
        train_labels_filtered.append(label)

train_dataset = PoemSentimentDataset(train_texts_filtered, train_labels_filtered, wp, method='avg', dimension=15)
valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp, dimension=15, method='avg')
test_dataset = PoemSentimentDataset(test_texts, test_labels, wp, dimension=15, method='avg')

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

bert_sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

bert_sensepolar_model.test_model(test_loader, loss_fn, device)

Epoch 1/1000 - Training Loss: 1.053638738281322
Epoch 1/1000 - Validation Loss: 1.0171215789658683
Epoch 2/1000 - Training Loss: 0.9925639100794522
Epoch 2/1000 - Validation Loss: 0.9635111263820103
Epoch 3/1000 - Training Loss: 0.9473707271072099
Epoch 3/1000 - Validation Loss: 0.9255195600645882
Epoch 4/1000 - Training Loss: 0.9190122473914668
Epoch 4/1000 - Validation Loss: 0.899821400642395
Epoch 5/1000 - Training Loss: 0.8953099430732008
Epoch 5/1000 - Validation Loss: 0.883085983140128
Epoch 6/1000 - Training Loss: 0.8806973864447396
Epoch 6/1000 - Validation Loss: 0.869168358189719
Epoch 7/1000 - Training Loss: 0.8698186525758708
Epoch 7/1000 - Validation Loss: 0.8581922820636204
Epoch 8/1000 - Training Loss: 0.8692817069449514
Epoch 8/1000 - Validation Loss: 0.849789559841156
Epoch 9/1000 - Training Loss: 0.8591442456785238
Epoch 9/1000 - Validation Loss: 0.8417972496577671
Epoch 10/1000 - Training Loss: 0.8497995273122247
Epoch 10/1000 - Validation Loss: 0.8350812111582074
Epo

In [12]:
bert_sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=5, model_name='sense_polar_bert_5dim_avg_projection.pth')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_sensepolar_model.to(device)

optimizer = torch.optim.AdamW(bert_sensepolar_model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 1000
patience = 10

preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]
valid_labels = dataset["validation"]["label"]

train_texts_filtered = []
train_labels_filtered = []
for text, label in zip(train_texts, train_labels):
    if label != 3:
        train_texts_filtered.append(text)
        train_labels_filtered.append(label)

train_dataset = PoemSentimentDataset(train_texts_filtered, train_labels_filtered, wp, method='avg', dimension=5)
valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp, dimension=5, method='avg')
test_dataset = PoemSentimentDataset(test_texts, test_labels, wp, dimension=5, method='avg')

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

bert_sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

bert_sensepolar_model.test_model(test_loader, loss_fn, device)

Epoch 1/1000 - Training Loss: 1.1926525426360797
Epoch 1/1000 - Validation Loss: 1.2067703008651733
Epoch 2/1000 - Training Loss: 1.141754723944754
Epoch 2/1000 - Validation Loss: 1.1599708965846471
Epoch 3/1000 - Training Loss: 1.104758622511378
Epoch 3/1000 - Validation Loss: 1.12150924546378
Epoch 4/1000 - Training Loss: 1.0683779210414526
Epoch 4/1000 - Validation Loss: 1.0902571507862635
Epoch 5/1000 - Training Loss: 1.0408909230862025
Epoch 5/1000 - Validation Loss: 1.064001100403922
Epoch 6/1000 - Training Loss: 1.0148555107836454
Epoch 6/1000 - Validation Loss: 1.0408532193728857
Epoch 7/1000 - Training Loss: 0.9974442922844077
Epoch 7/1000 - Validation Loss: 1.0214155656950814
Epoch 8/1000 - Training Loss: 0.9786445228558667
Epoch 8/1000 - Validation Loss: 1.0038565312113081
Epoch 9/1000 - Training Loss: 0.9645148706886003
Epoch 9/1000 - Validation Loss: 0.9892914635794503
Epoch 10/1000 - Training Loss: 0.953016644378878
Epoch 10/1000 - Validation Loss: 0.9765200785228184
Epoc

In [13]:
bert_sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=5, model_name='sense_polar_bert_5dim_cls_projection.pth')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_sensepolar_model.to(device)

optimizer = torch.optim.AdamW(bert_sensepolar_model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 1000
patience = 10

preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]
valid_labels = dataset["validation"]["label"]

train_texts_filtered = []
train_labels_filtered = []
for text, label in zip(train_texts, train_labels):
    if label != 3:
        train_texts_filtered.append(text)
        train_labels_filtered.append(label)

train_dataset = PoemSentimentDataset(train_texts_filtered, train_labels_filtered, wp, method='cls', dimension=5)
valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp, dimension=5, method='cls')
test_dataset = PoemSentimentDataset(test_texts, test_labels, wp, dimension=5, method='cls')

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

bert_sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

bert_sensepolar_model.test_model(test_loader, loss_fn, device)

Epoch 1/1000 - Training Loss: 0.9560141945784947
Epoch 1/1000 - Validation Loss: 0.9908946922847203
Epoch 2/1000 - Training Loss: 0.9376665679913647
Epoch 2/1000 - Validation Loss: 0.9753887483051845
Epoch 3/1000 - Training Loss: 0.9240813277802378
Epoch 3/1000 - Validation Loss: 0.9636855466025216
Epoch 4/1000 - Training Loss: 0.9334739086762914
Epoch 4/1000 - Validation Loss: 0.9566497206687927
Epoch 5/1000 - Training Loss: 0.9200524012997465
Epoch 5/1000 - Validation Loss: 0.9515175989695958
Epoch 6/1000 - Training Loss: 0.917256310301007
Epoch 6/1000 - Validation Loss: 0.9470119391168866
Epoch 7/1000 - Training Loss: 0.9084679023274835
Epoch 7/1000 - Validation Loss: 0.9434473770005363
Epoch 8/1000 - Training Loss: 0.9128536120900568
Epoch 8/1000 - Validation Loss: 0.9401130165372577
Epoch 9/1000 - Training Loss: 0.9037009522599994
Epoch 9/1000 - Validation Loss: 0.9372374159949166
Epoch 10/1000 - Training Loss: 0.9065343197786583
Epoch 10/1000 - Validation Loss: 0.9341982007026672

In [4]:
with io.capture_output() as captured:
    dataset = load_dataset("poem_sentiment")
    embedding_model_name = 'bert-base-uncased'
    out_path = './antonyms/'
    antonym_path = "data/polars.xlsx"
    embed_model = BERTWordEmbeddings()

    dictionary = Dictionary('wordnet', api_key='')    
    lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
    lookupSpace.create_lookup_files()
    antonym_path = out_path + "polar_dimensions.pkl"

    pdc = PolarDimensions(embed_model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
    pdc.create_polar_dimensions(out_path)

    wp = WordPolarity(embed_model, antonym_path=antonym_path, method='base-change')
    num_classes = 3
    bert_sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=39)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_sensepolar_model.to(device)

optimizer = torch.optim.AdamW(bert_sensepolar_model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 1000
patience = 10

preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]
valid_labels = dataset["validation"]["label"]

train_texts_filtered = []
train_labels_filtered = []
for text, label in zip(train_texts, train_labels):
    if label != 3:
        train_texts_filtered.append(text)
        train_labels_filtered.append(label)

train_dataset = PoemSentimentDataset(train_texts_filtered, train_labels_filtered, wp, method='avg', dimension=39)
valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp, dimension=39, method='avg')
test_dataset = PoemSentimentDataset(test_texts, test_labels, wp, dimension=39, method='avg')

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

bert_sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

bert_sensepolar_model.test_model(test_loader, loss_fn, device)

Epoch 1/1000 - Training Loss: 1.0903408414912674
Epoch 1/1000 - Validation Loss: 1.074751615524292
Epoch 2/1000 - Training Loss: 1.0620879317229648
Epoch 2/1000 - Validation Loss: 1.047755173274449
Epoch 3/1000 - Training Loss: 1.0367296731696938
Epoch 3/1000 - Validation Loss: 1.0240690197263445
Epoch 4/1000 - Training Loss: 1.0140067698820583
Epoch 4/1000 - Validation Loss: 1.0028311695371355
Epoch 5/1000 - Training Loss: 0.9945317979128856
Epoch 5/1000 - Validation Loss: 0.9837985209056309
Epoch 6/1000 - Training Loss: 0.9760339462532187
Epoch 6/1000 - Validation Loss: 0.9684718336377826
Epoch 7/1000 - Training Loss: 0.9614359803919522
Epoch 7/1000 - Validation Loss: 0.9540803943361554
Epoch 8/1000 - Training Loss: 0.9486970957720054
Epoch 8/1000 - Validation Loss: 0.941496661731175
Epoch 9/1000 - Training Loss: 0.9361916499317817
Epoch 9/1000 - Validation Loss: 0.9314370921679905
Epoch 10/1000 - Training Loss: 0.9280415514730057
Epoch 10/1000 - Validation Loss: 0.9227149827139718
E

In [ ]:
with io.capture_output() as captured:
    dataset = load_dataset("poem_sentiment")
    embedding_model_name = 'bert-base-uncased'
    out_path = './antonyms/'
    antonym_path = "data/polars.xlsx"
    embed_model = BERTWordEmbeddings()

    dictionary = Dictionary('wordnet', api_key='')    
    lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
    lookupSpace.create_lookup_files()
    antonym_path = out_path + "polar_dimensions.pkl"

    pdc = PolarDimensions(embed_model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
    pdc.create_polar_dimensions(out_path)

    wp = WordPolarity(embed_model, antonym_path=antonym_path, method='base-change')
    num_classes = 3
    bert_sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=39)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_sensepolar_model.to(device)

optimizer = torch.optim.AdamW(bert_sensepolar_model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 1000
patience = 10

preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]
valid_labels = dataset["validation"]["label"]

train_texts_filtered = []
train_labels_filtered = []
for text, label in zip(train_texts, train_labels):
    if label != 3:
        train_texts_filtered.append(text)
        train_labels_filtered.append(label)

train_dataset = PoemSentimentDataset(train_texts_filtered, train_labels_filtered, wp, method='avg', dimension=39)
valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp, dimension=39, method='avg')
test_dataset = PoemSentimentDataset(test_texts, test_labels, wp, dimension=39, method='avg')

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

bert_sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

bert_sensepolar_model.test_model(test_loader, loss_fn, device)

## SensePOLAR-Bert-wordnet-768-cls-antonym-pairs

In [11]:
with io.capture_output() as captured:
    dataset = load_dataset("poem_sentiment")
    embedding_model_name = 'bert-base-uncased'
    out_path = './antonyms/'
    antonym_path = "data/polars_all_combined.xlsx"
    embed_model = BERTWordEmbeddings()

    dictionary = Dictionary('wordnet', api_key='')    
    lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
    lookupSpace.create_lookup_files()
    antonym_path = out_path + "polar_dimensions.pkl"

    pdc = PolarDimensions(embed_model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
    pdc.create_polar_dimensions(out_path)

    wp = WordPolarity(embed_model, antonym_path=antonym_path, method='projection')
    num_classes = 3
    bert_sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=768, model_name='sense_polar_bert_768dim_cls_projection.pth')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_sensepolar_model.to(device)

optimizer = torch.optim.AdamW(bert_sensepolar_model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 1000
patience = 10

preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]
valid_labels = dataset["validation"]["label"]

train_texts_filtered = []
train_labels_filtered = []
for text, label in zip(train_texts, train_labels):
    if label != 3:
        train_texts_filtered.append(text)
        train_labels_filtered.append(label)

train_dataset = PoemSentimentDataset(train_texts_filtered, train_labels_filtered, wp, method='cls', dimension=768)
valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp, dimension=768, method='cls')
test_dataset = PoemSentimentDataset(test_texts, test_labels, wp, dimension=768, method='cls')

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

bert_sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

bert_sensepolar_model.test_model(test_loader, loss_fn, device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/1000 - Training Loss: 0.8786296580197677
Epoch 1/1000 - Validation Loss: 0.7586648889950344
Epoch 2/1000 - Training Loss: 0.7393391509101076
Epoch 2/1000 - Validation Loss: 0.6712460560458047
Epoch 3/1000 - Training Loss: 0.6869517806565987
Epoch 3/1000 - Validation Loss: 0.651786778654371
Epoch 4/1000 - Training Loss: 0.6917280419817511
Epoch 4/1000 - Validation Loss: 0.7006902098655701
Epoch 5/1000 - Training Loss: 0.6706293264649948
Epoch 5/1000 - Validation Loss: 0.7300385236740112
Epoch 6/1000 - Training Loss: 0.6325715847735135
Epoch 6/1000 - Validation Loss: 0.6170928009918758
Epoch 7/1000 - Training Loss: 0.6205803397691475
Epoch 7/1000 - Validation Loss: 0.6898888264383588
Epoch 8/1000 - Training Loss: 0.6184337909491557
Epoch 8/1000 - Validation Loss: 0.6519668187413897
Epoch 9/1000 - Training Loss: 0.5962813049554825
Epoch 9/1000 - Validation Loss: 0.6332175305911473
Epoch 10/1000 - Training Loss: 0.5818608391959712
Epoch 10/1000 - Validation Loss: 0.6227937085287911

## SensePOLAR-Bert-wordnet-768-avg-antonym-pairs-projection

In [34]:
with io.capture_output() as captured:
    dataset = load_dataset("poem_sentiment")
    embedding_model_name = 'bert-base-uncased'
    out_path = './antonyms/'
    antonym_path = "data/polars_all_combined.xlsx"
    embed_model = BERTWordEmbeddings()

    dictionary = Dictionary('wordnet', api_key='')    
    lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
    lookupSpace.create_lookup_files()
    antonym_path = out_path + "polar_dimensions.pkl"

    pdc = PolarDimensions(embed_model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
    pdc.create_polar_dimensions(out_path)

    wp = WordPolarity(embed_model, antonym_path=antonym_path, method='projection')
    num_classes = 3
    bert_sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=768, model_name='sense_polar_bert_768dim_avg_projection.pth')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_sensepolar_model.to(device)

optimizer = torch.optim.AdamW(bert_sensepolar_model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 1000
patience = 10

preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]
valid_labels = dataset["validation"]["label"]

train_texts_filtered = []
train_labels_filtered = []
for text, label in zip(train_texts, train_labels):
    if label != 3:
        train_texts_filtered.append(text)
        train_labels_filtered.append(label)

train_dataset = PoemSentimentDataset(train_texts_filtered, train_labels_filtered, wp, method='avg', dimension=768)
valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp, dimension=768, method='avg')
test_dataset = PoemSentimentDataset(test_texts, test_labels, wp, dimension=768, method='avg')

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

bert_sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

bert_sensepolar_model.test_model(test_loader, loss_fn, device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/1000 - Training Loss: 0.8364533984436179
Epoch 1/1000 - Validation Loss: 0.7601958640984127
Epoch 2/1000 - Training Loss: 0.6422438104197664
Epoch 2/1000 - Validation Loss: 0.7136863980974469
Epoch 3/1000 - Training Loss: 0.568898119454114
Epoch 3/1000 - Validation Loss: 0.6907659939357212
Epoch 4/1000 - Training Loss: 0.5132429031830914
Epoch 4/1000 - Validation Loss: 0.6744183089051928
Epoch 5/1000 - Training Loss: 0.4793102128325768
Epoch 5/1000 - Validation Loss: 0.6845889346940177
Epoch 6/1000 - Training Loss: 0.4581925357287785
Epoch 6/1000 - Validation Loss: 0.6623368390968868
Epoch 7/1000 - Training Loss: 0.4229476423758381
Epoch 7/1000 - Validation Loss: 0.6752352799688067
Epoch 8/1000 - Training Loss: 0.39885520935058594
Epoch 8/1000 - Validation Loss: 0.660498423235757
Epoch 9/1000 - Training Loss: 0.37579926370449784
Epoch 9/1000 - Validation Loss: 0.6725385785102844
Epoch 10/1000 - Training Loss: 0.35883785531205953
Epoch 10/1000 - Validation Loss: 0.66586345859936

## SensePOLAR-Bert-wordnet-768-avg-antonym-pairs-base-change

In [10]:
with io.capture_output() as captured:
    dataset = load_dataset("poem_sentiment")
    embedding_model_name = 'bert-base-uncased'
    out_path = './antonyms/'
    antonym_path = "data/polars_all_combined.xlsx"
    embed_model = BERTWordEmbeddings()

    dictionary = Dictionary('wordnet', api_key='')    
    lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
    lookupSpace.create_lookup_files()
    antonym_path = out_path + "polar_dimensions.pkl"

    pdc = PolarDimensions(embed_model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
    pdc.create_polar_dimensions(out_path)

    wp = WordPolarity(embed_model, antonym_path=antonym_path, method='base-change')
    num_classes = 3
    bert_sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=768, model_name='sense_polar_bert_768dim_avg_base.pth')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_sensepolar_model.to(device)

optimizer = torch.optim.AdamW(bert_sensepolar_model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 1000
patience = 10

preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]
valid_labels = dataset["validation"]["label"]

train_texts_filtered = []
train_labels_filtered = []
for text, label in zip(train_texts, train_labels):
    if label != 3:
        train_texts_filtered.append(text)
        train_labels_filtered.append(label)

train_dataset = PoemSentimentDataset(train_texts_filtered, train_labels_filtered, wp, method='avg', dimension=768)
valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp, dimension=768, method='avg')
test_dataset = PoemSentimentDataset(test_texts, test_labels, wp, dimension=768, method='avg')

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

bert_sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

bert_sensepolar_model.test_model(test_loader, loss_fn, device)

Epoch 1/1000 - Training Loss: 1.0827747201019864
Epoch 1/1000 - Validation Loss: 1.0673840386526925
Epoch 2/1000 - Training Loss: 1.0535059357589145
Epoch 2/1000 - Validation Loss: 1.0398616535323006
Epoch 3/1000 - Training Loss: 1.0274418671176118
Epoch 3/1000 - Validation Loss: 1.0166359288351876
Epoch 4/1000 - Training Loss: 1.006518820546708
Epoch 4/1000 - Validation Loss: 0.9949518953050885
Epoch 5/1000 - Training Loss: 0.9862614487701992
Epoch 5/1000 - Validation Loss: 0.9772100789206368
Epoch 6/1000 - Training Loss: 0.9696276986374045
Epoch 6/1000 - Validation Loss: 0.9616480469703674
Epoch 7/1000 - Training Loss: 0.9547595977783203
Epoch 7/1000 - Validation Loss: 0.9482990843909127
Epoch 8/1000 - Training Loss: 0.9418248489217939
Epoch 8/1000 - Validation Loss: 0.9367763655526298
Epoch 9/1000 - Training Loss: 0.931747315064916
Epoch 9/1000 - Validation Loss: 0.9268477559089661
Epoch 10/1000 - Training Loss: 0.9229873958623634
Epoch 10/1000 - Validation Loss: 0.9185912268502372


## SensePOLAR-Bert-wordnet-1586-cls-antonym-pairs

In [4]:
with io.capture_output() as captured:
    dataset = load_dataset("poem_sentiment")
    embedding_model_name = 'bert-base-uncased'
    out_path = './antonyms/'
    antonym_path = "data/polars_all_combined.xlsx"
    embed_model = BERTWordEmbeddings()

    dictionary = Dictionary('wordnet', api_key='')    
    lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
    lookupSpace.create_lookup_files()
    antonym_path = out_path + "polar_dimensions.pkl"

    pdc = PolarDimensions(embed_model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
    pdc.create_polar_dimensions(out_path)

    wp = WordPolarity(embed_model, antonym_path=antonym_path, method='projection')
    num_classes = 3
    bert_sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=1586, model_name='sense_polar_bert_1586dim_avg_projection.pth')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_sensepolar_model.to(device)

optimizer = torch.optim.AdamW(bert_sensepolar_model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 1000
patience = 10

preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]
valid_labels = dataset["validation"]["label"]

train_texts_filtered = []
train_labels_filtered = []
for text, label in zip(train_texts, train_labels):
    if label != 3:
        train_texts_filtered.append(text)
        train_labels_filtered.append(label)

train_dataset = PoemSentimentDataset(train_texts_filtered, train_labels_filtered, wp, method='avg', dimension=1586)
valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp, dimension=1586, method='avg')
test_dataset = PoemSentimentDataset(test_texts, test_labels, wp, dimension=1586, method='avg')

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

bert_sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

bert_sensepolar_model.test_model(test_loader, loss_fn, device)

Some weights of the model checkpoint at dbmdz/bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/1000 - Training Loss: 1.017482359454317
Epoch 1/1000 - Validation Loss: 0.9389297664165497
Epoch 2/1000 - Training Loss: 0.8737683217480497
Epoch 2/1000 - Validation Loss: 0.88240099804742
Epoch 3/1000 - Training Loss: 0.8088448064507179
Epoch 3/1000 - Validation Loss: 0.9196915115628924
Epoch 4/1000 - Training Loss: 0.8188239903944843
Epoch 4/1000 - Validation Loss: 1.0222078987530299
Epoch 5/1000 - Training Loss: 0.8790274481728392
Epoch 5/1000 - Validation Loss: 1.0034197228295463
Epoch 6/1000 - Training Loss: 0.9122673734179083
Epoch 6/1000 - Validation Loss: 1.0593508567128862
Epoch 7/1000 - Training Loss: 0.7878658147353046
Epoch 7/1000 - Validation Loss: 0.9403156212397984
Epoch 8/1000 - Training Loss: 0.667004295677509
Epoch 8/1000 - Validation Loss: 0.9154653208596366
Epoch 9/1000 - Training Loss: 0.6413853196602948
Epoch 9/1000 - Validation Loss: 0.9654999205044338
Epoch 10/1000 - Training Loss: 0.6739068197191886
Epoch 10/1000 - Validation Loss: 0.9529091886111668
Ep

## SensePOLAR-Albert-wordnet-786-avg-antonym-pairs-last-2nd-layer-base-change

In [5]:
with io.capture_output() as captured:
    dataset = load_dataset("poem_sentiment")
    embedding_model_name = 'bert-base-uncased'
    out_path = './antonyms/'
    antonym_path = "data/polars_all_combined.xlsx"
    embed_model = ALBERTWordEmbeddings()

    dictionary = Dictionary('wordnet', api_key='')    
    lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
    lookupSpace.create_lookup_files()
    antonym_path = out_path + "polar_dimensions.pkl"

    pdc = PolarDimensions(embed_model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
    pdc.create_polar_dimensions(out_path,"/polar_dimensions.pkl" )

    wp = WordPolarity(embed_model, antonym_path=antonym_path, method='base-change')
    num_classes = 3
    bert_sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=768, model_name='sense_polar_albert_768dim_avg_base_2.pth')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_sensepolar_model.to(device)

optimizer = torch.optim.AdamW(bert_sensepolar_model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 1000
patience = 10

preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]
valid_labels = dataset["validation"]["label"]

train_texts_filtered = []
train_labels_filtered = []
for text, label in zip(train_texts, train_labels):
    if label != 3:
        train_texts_filtered.append(text)
        train_labels_filtered.append(label)

train_dataset = PoemSentimentDataset(train_texts_filtered, train_labels_filtered, wp, method='avg', dimension=768)
valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp, dimension=768, method='avg')
test_dataset = PoemSentimentDataset(test_texts, test_labels, wp, dimension=768, method='avg')

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

bert_sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

bert_sensepolar_model.test_model(test_loader, loss_fn, device)

Epoch 1/1000 - Training Loss: 1.8424459160368443e+18
Epoch 1/1000 - Validation Loss: 8.216748810477802e+17
Epoch 2/1000 - Training Loss: 1.598961050502814e+18
Epoch 2/1000 - Validation Loss: 1.4078033887590013e+18
Epoch 3/1000 - Training Loss: 1.405448766085349e+18
Epoch 3/1000 - Validation Loss: 2.1277669502469501e+18
Epoch 4/1000 - Training Loss: 1.783240355879831e+18
Epoch 4/1000 - Validation Loss: 2.4419793359188224e+18
Epoch 5/1000 - Training Loss: 2.406814631677642e+18
Epoch 5/1000 - Validation Loss: 3.0406097276973875e+17
Epoch 6/1000 - Training Loss: 1.3402520722547645e+18
Epoch 6/1000 - Validation Loss: 1.8053268307108168e+18
Epoch 7/1000 - Training Loss: 1.627317748737527e+18
Epoch 7/1000 - Validation Loss: 1.0866283898810438e+18
Epoch 8/1000 - Training Loss: 1.448453831481677e+18
Epoch 8/1000 - Validation Loss: 1.4877117903312932e+18
Epoch 9/1000 - Training Loss: 1.5245320672576625e+18
Epoch 9/1000 - Validation Loss: 1.1229758342050986e+18
Epoch 10/1000 - Training Loss: 1.48

## SensePOLAR-Albert-wordnet-786-avg-antonym-pairs-last-2nd-layer-projection

In [6]:
with io.capture_output() as captured:
    wp = WordPolarity(embed_model, antonym_path=antonym_path, method='projection')
    num_classes = 3
    bert_sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=768, model_name='sense_polar_albert_768dim_avg_projection_2.pth')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_sensepolar_model.to(device)

optimizer = torch.optim.AdamW(bert_sensepolar_model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 1000
patience = 10

preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]
valid_labels = dataset["validation"]["label"]

train_texts_filtered = []
train_labels_filtered = []
for text, label in zip(train_texts, train_labels):
    if label != 3:
        train_texts_filtered.append(text)
        train_labels_filtered.append(label)

train_dataset = PoemSentimentDataset(train_texts_filtered, train_labels_filtered, wp, method='avg', dimension=768)
valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp, dimension=768, method='avg')
test_dataset = PoemSentimentDataset(test_texts, test_labels, wp, dimension=768, method='avg')

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

bert_sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

bert_sensepolar_model.test_model(test_loader, loss_fn, device)

Epoch 1/1000 - Training Loss: 1.9177725649508774e+18
Epoch 1/1000 - Validation Loss: 1.0923075490542668e+18
Epoch 2/1000 - Training Loss: 1.5920722049692035e+18
Epoch 2/1000 - Validation Loss: 6.963909255416288e+17
Epoch 3/1000 - Training Loss: 1.7330437952981898e+18
Epoch 3/1000 - Validation Loss: 1.382564158245812e+18
Epoch 4/1000 - Training Loss: 1.685140821158465e+18
Epoch 4/1000 - Validation Loss: 1.1655928852635602e+18
Epoch 5/1000 - Training Loss: 1.7927917667129408e+18
Epoch 5/1000 - Validation Loss: 1.3966667299933015e+18
Epoch 6/1000 - Training Loss: 1.5484662201636667e+18
Epoch 6/1000 - Validation Loss: 8.702361128500312e+17
Epoch 7/1000 - Training Loss: 1.3937411597898207e+18
Epoch 7/1000 - Validation Loss: 1.1795462572099876e+18
Epoch 8/1000 - Training Loss: 1.3745045464887718e+18
Epoch 8/1000 - Validation Loss: 1.0153677503404255e+18
Epoch 9/1000 - Training Loss: 1.4115150686557926e+18
Epoch 9/1000 - Validation Loss: 2.42293875928031e+18
Epoch 10/1000 - Training Loss: 1.4

## SensePOLAR-roberta-wordnet-786-avg-antonym-pairs-last-2nd-layer-base-change

In [12]:
with io.capture_output() as captured:
    dataset = load_dataset("poem_sentiment")
    embedding_model_name = 'bert-base-uncased'
    out_path = './antonyms/'
    antonym_path = "data/polars_all_combined.xlsx"
    embed_model = RoBERTaWordEmbeddings()

    dictionary = Dictionary('wordnet', api_key='')    
    lookupSpace = LookupCreator(dictionary, out_path, antonyms_file_path=antonym_path)
    lookupSpace.create_lookup_files()
    antonym_path = out_path + "polar_dimensions.pkl"

    pdc = PolarDimensions(embed_model, antonym_path=out_path + "antonym_wordnet_example_sentences_readable_extended.txt")
    pdc.create_polar_dimensions(out_path,"/polar_dimensions.pkl" )

    wp = WordPolarity(embed_model, antonym_path=antonym_path, method='base-change')
    num_classes = 3
    bert_sensepolar_model = PolarEmbeddingClassifier(num_classes=num_classes, polar_dimension=768, model_name='sense_polar_albert_768dim_avg_base_2.pth')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bert_sensepolar_model.to(device)

optimizer = torch.optim.AdamW(bert_sensepolar_model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

num_epochs = 1000
patience = 10

preprocess_text = lambda verse: re.sub(r'\W+', ' ', re.sub(r'_([^_]+)_', r'\1', verse))
train_texts = [preprocess_text(verse) for verse in dataset["train"]["verse_text"]]
test_texts = [preprocess_text(verse) for verse in dataset["test"]["verse_text"]]
valid_texts = [preprocess_text(verse) for verse in dataset["validation"]["verse_text"]]
train_labels = dataset["train"]["label"]
test_labels = dataset["test"]["label"]
valid_labels = dataset["validation"]["label"]

train_texts_filtered = []
train_labels_filtered = []
for text, label in zip(train_texts, train_labels):
    if label != 3:
        train_texts_filtered.append(text)
        train_labels_filtered.append(label)

train_dataset = PoemSentimentDataset(train_texts_filtered, train_labels_filtered, wp, method='avg', dimension=768)
valid_dataset = PoemSentimentDataset(valid_texts, valid_labels, wp, dimension=768, method='avg')
test_dataset = PoemSentimentDataset(test_texts, test_labels, wp, dimension=768, method='avg')

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)  
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

bert_sensepolar_model.train_model(train_loader, valid_loader, num_epochs, patience, optimizer, loss_fn, device)

bert_sensepolar_model.test_model(test_loader, loss_fn, device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/1000 - Training Loss: 1.0753969831286736
Epoch 1/1000 - Validation Loss: 1.0603592566081457
Epoch 2/1000 - Training Loss: 1.047644262043935
Epoch 2/1000 - Validation Loss: 1.0346036383083888
Epoch 3/1000 - Training Loss: 1.022906316901153
Epoch 3/1000 - Validation Loss: 1.011220165661403
Epoch 4/1000 - Training Loss: 1.0020200272776045
Epoch 4/1000 - Validation Loss: 0.9909040757587978
Epoch 5/1000 - Training Loss: 0.9824208099887056
Epoch 5/1000 - Validation Loss: 0.9739036900656564
Epoch 6/1000 - Training Loss: 0.9649728963959892
Epoch 6/1000 - Validation Loss: 0.9585876720292228
Epoch 7/1000 - Training Loss: 0.9518143298490992
Epoch 7/1000 - Validation Loss: 0.945563691002982
Epoch 8/1000 - Training Loss: 0.9397439034479969
Epoch 8/1000 - Validation Loss: 0.9340812478746686
Epoch 9/1000 - Training Loss: 0.9290653231008997
Epoch 9/1000 - Validation Loss: 0.9245833839688983
Epoch 10/1000 - Training Loss: 0.9209543410337196
Epoch 10/1000 - Validation Loss: 0.916976239000048
Epo